<img src="https://s3.amazonaws.com/datascienceheroes.com/EDV/360_banner_python.png" width="400px">


# Escuela de Datos Vivos 
## LAB P.F.5: Modelos de regresión
Creado por Pablo Casas | https://escueladedatosvivos.ai

Resuelto por Pablo Sotomayor

---

## Ejercicios

1) Modificar los parámetros del random forest a : `min_samples_leaf=20`, `min_samples_split=40`. Pueden también variarlos. 

2) Comparar el R2 del modelo anterior, y de este que posee la modificación de sus parámetros. 

3) Probar otro parámetros: `max_depth`.

4) ¿Qué pasa si min_samples_split < min_samples_leaf?


## Solución

In [146]:
# Librerías que utilizaremos

import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from minepy import MINE
from __future__ import division
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import accuracy_score, explained_variance_score
from yellowbrick.regressor import PredictionError
from yellowbrick.regressor import ResidualsPlot
from funpymodeling.exploratory import freq_tbl, status, profiling_num, cat_vars, num_vars
from plotnine import *

%matplotlib inline

warnings.filterwarnings('ignore')

1) Carga de datos

In [45]:
data=pd.read_csv("../data/hotels.csv", sep = ",")

print(f"Cantidad de registros: {data.shape[0]}")
print(f"Cantidad de variables: {data.shape[1]}")

Cantidad de registros: 119390
Cantidad de variables: 32


Inspeccionamos los datos elegiendo una muestra al azar

In [46]:
data.sample(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
56787,City Hotel,1,41,2016,September,38,14,0,2,2,...,No Deposit,86.0,NaN,0,Transient-Party,130.00,0,0,Canceled,2016-08-10
107534,City Hotel,0,3,2017,March,10,11,2,1,2,...,No Deposit,9.0,NaN,0,Transient,98.00,0,1,Check-Out,2017-03-14
115901,City Hotel,0,33,2017,July,28,10,1,2,1,...,No Deposit,10.0,NaN,0,Transient,100.00,0,0,Check-Out,2017-07-13
79490,City Hotel,0,5,2015,October,44,27,0,1,1,...,No Deposit,NaN,40.0,0,Transient,65.00,0,0,Check-Out,2015-10-28
39175,Resort Hotel,0,199,2017,August,31,3,2,5,3,...,No Deposit,250.0,NaN,0,Transient,187.37,0,2,Check-Out,2017-08-10


Seleccionamos algunos atributos de interés

In [47]:
variables = ['total_of_special_requests','deposit_type','customer_type', 'stays_in_weekend_nights', 
             'stays_in_week_nights', 'required_car_parking_spaces','arrival_date_month',
             'arrival_date_day_of_month','hotel']

data2 = data[variables].copy()

Revisamos los datos para el nuevo conjunto de variables

In [48]:
data2.head()

,total_of_special_requests,deposit_type,customer_type,stays_in_weekend_nights,stays_in_week_nights,required_car_parking_spaces,arrival_date_month,arrival_date_day_of_month,hotel
0,0,No Deposit,Transient,0,0,0,July,1,Resort Hotel
1,0,No Deposit,Transient,0,0,0,July,1,Resort Hotel
2,0,No Deposit,Transient,0,1,0,July,1,Resort Hotel
3,0,No Deposit,Transient,0,1,0,July,1,Resort Hotel
4,1,No Deposit,Transient,0,2,0,July,1,Resort Hotel


Revisamos el estado de los datos

In [49]:
status(data2).sort_values(by=["type", "p_nan", "p_zeros", "unique"], ascending=False)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
6,arrival_date_month,0,0.0,0,0.000000,12,object
2,customer_type,0,0.0,0,0.000000,4,object
1,deposit_type,0,0.0,0,0.000000,3,object
8,hotel,0,0.0,0,0.000000,2,object
5,required_car_parking_spaces,0,0.0,111974,0.937884,5,int64
0,total_of_special_requests,0,0.0,70318,0.588977,6,int64
3,stays_in_weekend_nights,0,0.0,51998,0.435531,17,int64
4,stays_in_week_nights,0,0.0,7645,0.064034,35,int64
7,arrival_date_day_of_month,0,0.0,0,0.000000,31,int64


Tenemos 4 variables declaradas como categóricas y 5 declaradas como numéricas. El dataset no tiene valores nulos, pero hay datos con valores cero (pero en ningún caso valores imposibles). La variable "arrival_date_day_of_month" está mal declaradas como numérica.

En este problema la variable a predecir is "total_stay", la cantidad de noches que permanece un viajero en el hotel. Creamos la variable a predecir a partir de las variables "stays_in_weekend_nights" (cantidad de noches en fines de semana) y "stays_in_week_nights" (cantidad de noches en días de semana)

In [50]:
data_y = data2['stays_in_week_nights'] + data2['stays_in_weekend_nights']
data_y = data_y.rename("total_stays")

data_x = data2.drop(['stays_in_week_nights','stays_in_weekend_nights'], axis=1)

Inspeccionamos las variables independientes y la variable a predecir

In [51]:
data_x.head()

,total_of_special_requests,deposit_type,customer_type,required_car_parking_spaces,arrival_date_month,arrival_date_day_of_month,hotel
0,0,No Deposit,Transient,0,July,1,Resort Hotel
1,0,No Deposit,Transient,0,July,1,Resort Hotel
2,0,No Deposit,Transient,0,July,1,Resort Hotel
3,0,No Deposit,Transient,0,July,1,Resort Hotel
4,1,No Deposit,Transient,0,July,1,Resort Hotel


In [52]:
data_y.head()

0    0
1    0
2    1
3    1
4    2
Name: total_stays, dtype: int64

Transformamos la variable "arrival_date_day_of_month" a categórica

In [53]:
data_x['arrival_date_day_of_month'] = data_x['arrival_date_day_of_month'].astype('str')

Hacemos un análisis exploratorio de los datos. Empezamos con las variables categóricas

Variable: "total_of_special_requests"

In [54]:
freq_tbl(data_x["deposit_type"])

,deposit_type,frequency,percentage,cumulative_perc
0,No Deposit,104641,0.876464,0.876464
1,Non Refund,14587,0.122179,0.998643
2,Refundable,162,0.001357,1.000000


Vemos que las clases están desbalanceadas, la mayoría de los registros indican que no se hizo un depósito

Variable: "customer_type

In [55]:
freq_tbl(data_x["customer_type"])

,customer_type,frequency,percentage,cumulative_perc
0,Transient,89613,0.750591,0.750591
1,Transient-Party,25124,0.210436,0.961027
2,Contract,4076,0.034140,0.995167
3,Group,577,0.004833,1.000000


La mayoría de los clientes son transitorios, seguidos por los que ingresan debido a una fiesta.

Variable: "arrival_date_month"

In [56]:
freq_tbl(data_x["arrival_date_month"])

,arrival_date_month,frequency,percentage,cumulative_perc
0,August,13877,0.116233,0.116233
1,July,12661,0.106047,0.222280
2,May,11791,0.098760,0.321040
3,October,11160,0.093475,0.414515
4,April,11089,0.092880,0.507396
5,June,10939,0.091624,0.599020
6,September,10508,0.088014,0.687034
7,March,9794,0.082034,0.769068
8,February,8068,0.067577,0.836645
9,November,6794,0.056906,0.893551


En este caso hay una distribución aproximadamente uniforme para los meses en que los clientes se hospedan. Hay una leve tendencia hacia agosto y julio (meses del verano en el hemisferio norte).

Variable: "arrival_date_day_of_month"

In [57]:
freq_tbl(data_x["arrival_date_day_of_month"])

,arrival_date_day_of_month,frequency,percentage,cumulative_perc
0,17,4406,0.036904,0.036904
1,5,4317,0.036159,0.073063
2,15,4196,0.035145,0.108208
3,25,4160,0.034844,0.143052
4,26,4147,0.034735,0.177787
5,9,4096,0.034308,0.212095
6,12,4087,0.034232,0.246327
7,16,4078,0.034157,0.280484
8,2,4055,0.033964,0.314448
9,19,4052,0.033939,0.348388


Nuevamente vemos una distribución aproximadamente uniforme, esta vez para el día del mes en que eligen hospedarse los clientes. El día menos elegido es el 31.

Analizamos que es lo que ocurre cuando dividimos los días en categorías (elegimos 5 categorías)

In [58]:
data_x['arrival_date_day_of_month'] = data_x['arrival_date_day_of_month'].astype(int)

In [60]:
data_x["arrival_date_day_of_month"]

0          1
1          1
2          1
3          1
4          1
          ..
119385    30
119386    31
119387    31
119388    31
119389    29
Name: arrival_date_day_of_month, Length: 119390, dtype: int64

In [92]:
data_x["arrival_date_day_of_month_cat"], bins = pd.qcut(data_x["arrival_date_day_of_month"], q=5, retbins=True)
freq_tbl(data_x["arrival_date_day_of_month_cat"])

,arrival_date_day_of_month_cat,frequency,percentage,cumulative_perc
0,"(0.999, 7.0]",27114,0.227104,0.227104
1,"(13.0, 19.0]",24553,0.205654,0.432758
2,"(19.0, 25.0]",23164,0.194020,0.626778
3,"(7.0, 13.0]",23023,0.192839,0.819616
4,"(25.0, 31.0]",21536,0.180384,1.000000


Vemos que en la primera semana de los meses es donde los clientes eligen hospedarse con más frecuencia. La última semana del mes es la fecha menos frecuente.

Variable: "hotel"

In [74]:
freq_tbl(data_x["hotel"])

,hotel,frequency,percentage,cumulative_perc
0,City Hotel,79330,0.664461,0.664461
1,Resort Hotel,40060,0.335539,1.000000


Vemos que las clases no están muy desbalanceadas, aunque la mayoría de registros corresponden al hotel Resort Hotel.

Analizamos ahora las variables numéricas.

Variable: "total_of_special_requests"	

In [79]:
profiling_num(data_x["total_of_special_requests"])

,variable,mean,std_dev,variation_coef,p_0.01,p_0.05,p_0.25,p_0.5,p_0.75,p_0.95,p_0.99
0,total_of_special_requests,0.571363,0.792798,1.387557,0.0,0.0,0.0,0.0,1.0,2.0,3.0


In [81]:
data_x["total_of_special_requests"].value_counts()

0    70318
1    33226
2    12969
3     2497
4      340
5       40
Name: total_of_special_requests, dtype: int64

In [82]:
data_x["total_of_special_requests"].describe()

count    119390.000000
mean          0.571363
std           0.792798
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           5.000000
Name: total_of_special_requests, dtype: float64

Vemos que esta variable está desbalanceada, la mayoría de los casos corresponden a 0 pedidos especiales, seguido de 1 pedido especial. El coeficiente de variación es mayor a 1, lo que sugiere presencia de outliers. Esto se confirma, pues el cuantil 0.99 corresponde a 3 pedidos especiales, y hay 380 casos de 4 o 5 pedidos.

Variable: "required_car_parking_spaces"

In [84]:
profiling_num(data_x["required_car_parking_spaces"])

,variable,mean,std_dev,variation_coef,p_0.01,p_0.05,p_0.25,p_0.5,p_0.75,p_0.95,p_0.99
0,required_car_parking_spaces,0.062518,0.245291,3.923541,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [85]:
data_x["required_car_parking_spaces"].value_counts()

0    111974
1      7383
2        28
3         3
8         2
Name: required_car_parking_spaces, dtype: int64

In [86]:
data_x["required_car_parking_spaces"].describe()

count    119390.000000
mean          0.062518
std           0.245291
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           8.000000
Name: required_car_parking_spaces, dtype: float64

Nuevamente vemos que la clase está desbalanceada, en este caso es aún más significativa. Hay 33 casos que pueden tratarse como outliers.

Analizamos ahora la variable a predecir

In [93]:
profiling_num(data_y)

,variable,mean,std_dev,variation_coef,p_0.01,p_0.05,p_0.25,p_0.5,p_0.75,p_0.95,p_0.99
0,total_stays,3.4279,2.557439,0.746066,1.0,1.0,2.0,3.0,4.0,7.0,14.0


In [94]:
data_y.value_counts()

2     27643
3     27076
1     21020
4     17383
7      8655
5      7784
6      3857
8      1161
10     1139
14      916
9       841
0       715
11      396
12      223
13      142
15       75
21       71
16       40
25       37
28       35
18       35
19       22
17       20
22       14
29       14
20       14
30       13
23        8
26        6
24        6
27        5
35        5
42        4
33        3
56        2
57        1
34        1
38        1
43        1
69        1
45        1
46        1
48        1
49        1
60        1
Name: total_stays, dtype: int64

In [95]:
data_y.describe()

count    119390.000000
mean          3.427900
std           2.557439
min           0.000000
25%           2.000000
50%           3.000000
75%           4.000000
max          69.000000
Name: total_stays, dtype: float64

Vemos que la variable a predecir tiene un coeficiente de variación menor a 1, por lo que no está muy afectada por outliers (aunque si tiene una cantidad significativa de ellos). La cantidad de días que permanece un huesped en el hotel es de 3.4 $\pm$ 2.6 al 66% de precisión (1 $\sigma$)

Hacemos un análisis bivariado. Primero entre las variables categóricas y la variable a predecir (numérica). Hacemos un dataframe con las variables independientes y la variable a predecir.

In [121]:
data_xy = pd.concat([data_x, data_y], axis=1)
data_xy = data_xy.drop("arrival_date_day_of_month", axis=1)

Analizamos las variables "deposit_type" y "total_stays"

In [122]:
data_xy.groupby('deposit_type')['total_stays'].mean().sort_values(ascending=False)

deposit_type
Refundable    3.827160
No Deposit    3.526964
Non Refund    2.712826
Name: total_stays, dtype: float64

Vemos que los clientes que pagan un depósito reembolsable se hospedan en promedio más días.

Analizamos las variables "customer_type" y "total_stays"

In [123]:
data_xy.groupby('customer_type')['total_stays'].mean().sort_values(ascending=False)

customer_type
Contract           5.320658
Transient          3.447145
Transient-Party    3.064719
Group              2.882149
Name: total_stays, dtype: float64

Los clientes que se hospedan en la modalidad Contract se quedan más días en promedio.

Analizamos las variables "arrival_date_day_of_month" (categorizada) y "total_stays

In [124]:
data_xy.groupby('arrival_date_day_of_month_cat')['total_stays'].mean().sort_values(ascending=False)

arrival_date_day_of_month_cat
(0.999, 7.0]    3.604522
(25.0, 31.0]    3.403278
(19.0, 25.0]    3.383181
(7.0, 13.0]     3.373974
(13.0, 19.0]    3.347208
Name: total_stays, dtype: float64

Aunque es muy estable, en promedio los clientes que se hospedan la primera semana se alojan más días (como vimos previamente)

Analizamos las variables "hotel" y "total_stays

In [125]:
data_xy.groupby('hotel')['total_stays'].mean().sort_values(ascending=False)

hotel
Resort Hotel    4.318547
City Hotel      2.978142
Name: total_stays, dtype: float64

En promedio los clientes que se hospedan en el Resort Hotel se quedan más días.

Para las variables numéricas analizamos la correlación lineal con la estadística r (Pearson)

In [126]:
data_xy.corr()

,total_of_special_requests,required_car_parking_spaces,total_stays
total_of_special_requests,1.000000,0.082626,0.079259
required_car_parking_spaces,0.082626,1.000000,-0.025794
total_stays,0.079259,-0.025794,1.000000


Vemos hay una correlación muy débil entre las variables.

Convertimos todas las variables a numéricas haciendo one-hot encoding

In [128]:
data_xy_ohe = pd.get_dummies(data_xy, drop_first=True)
status(data_xy_ohe)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,total_of_special_requests,0,0.0,70318,0.588977,6,int64
1,required_car_parking_spaces,0,0.0,111974,0.937884,5,int64
2,total_stays,0,0.0,715,0.005989,45,int64
3,deposit_type_Non Refund,0,0.0,104803,0.877821,2,uint8
4,deposit_type_Refundable,0,0.0,119228,0.998643,2,uint8
5,customer_type_Group,0,0.0,118813,0.995167,2,uint8
6,customer_type_Transient,0,0.0,29777,0.249409,2,uint8
7,customer_type_Transient-Party,0,0.0,94266,0.789564,2,uint8
8,arrival_date_month_August,0,0.0,105513,0.883767,2,uint8
9,arrival_date_month_December,0,0.0,112610,0.943211,2,uint8


Obtenemos 24 variables numéricas sin redundancias

Separamos los datos en TS y TR

In [141]:
data_x_ohe = data_xy_ohe.drop('total_stays', axis=1)
data_y_ohe = data_xy_ohe['total_stays']

In [142]:
x_tr, x_ts, y_tr, y_ts = train_test_split(data_x_ohe, data_y_ohe, test_size=0.2, random_state=0)

Creamos un modelo de regresión lineal

In [143]:
lr = LinearRegression()

Entrenamos al modelo

In [144]:
lr.fit(x_tr, y_tr)

LinearRegression()

Obtenemos las predicciones para TR y TS

In [134]:
pred_tr = lr.predict(x_tr)
pred_ts = lr.predict(x_ts)

Para un modelo de regresión no podemos calcular el accuracy (solo tiene sentido calcularlo en los problemas de clasificación). En su lugar calculamos las métricas "explained_variance_score" y "score" (r2 score). Los valores van entre 0 y 1, siendo 1 el mejor caso.

In [148]:
explained_variance_score(y_ts, pred_ts)

0.11376638784823978

In [149]:
lr.score(x_ts,y_ts)

0.11374892775603684

Vemos que en ambos casos el score es muy bajo, del orden del 11%. Como este modelo no tiene parámetros no podemos optimizarlo.

Creamos ahora un modelo RandomForestRegressor

In [150]:
rf = RandomForestRegressor()

Para encontrar parámetros adecuados usaremos hyperparameter tuning con GridSearchCV. Para el parámetro "n_estimator" consideramos los valores 100, 200, 500, y 1000. Para "max_features" consideramos 2 y 3.

In [152]:
params = {
    'n_estimators' : [100, 200, 500, 1000],
    'max_features': [2,3]
}

grid_rf = GridSearchCV(estimator = rf,
                        param_grid = params,
                        scoring = 'neg_mean_absolute_error',  # 
                        cv = 5, 
                        verbose = 1,
                        n_jobs = -1) 

Hacemos los ajustes para todos los modelos definidos en el paso anterior. Este proceso puede tardar unos minutos

In [153]:
grid_rf.fit(x_tr, y_tr)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  8.6min finished


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_features': [2, 3],
                         'n_estimators': [100, 200, 500, 1000]},
             scoring='neg_mean_absolute_error', verbose=1)

Imprimirmos la mejor combinación de parámetros

In [154]:
grid_rf.best_estimator_

RandomForestRegressor(max_features=2)

Nos indica que el caso más óptimo corresponde a `max_features=2`. Usamos el menor número de n_estimators.

Ahora predecimos casos nuevos y de entrenamiento

In [158]:
pred_tr = grid_rf.predict(x_tr)
pred_ts = grid_rf.predict(x_ts)

Analizamos las métricas de error

In [159]:
pd.DataFrame(grid_rf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,9.821567,0.165244,0.428139,0.030359,2,100,"{'max_features': 2, 'n_estimators': 100}",-1.519357,-1.535575,-1.526275,-1.501750,-1.517911,-1.520174,0.011132,1
1,19.716498,0.094186,0.848896,0.048027,2,200,"{'max_features': 2, 'n_estimators': 200}",-1.519965,-1.535860,-1.526153,-1.501761,-1.517514,-1.520251,0.011207,4
2,51.448103,1.680089,2.674867,0.397245,2,500,"{'max_features': 2, 'n_estimators': 500}",-1.519963,-1.535494,-1.526474,-1.501695,-1.518007,-1.520327,0.011140,8
3,107.251406,2.484647,5.453024,0.769008,2,1000,"{'max_features': 2, 'n_estimators': 1000}",-1.519816,-1.535404,-1.526304,-1.502017,-1.517880,-1.520284,0.010996,6
4,11.817837,0.756276,0.574462,0.130136,3,100,"{'max_features': 3, 'n_estimators': 100}",-1.520172,-1.534965,-1.526526,-1.501870,-1.517748,-1.520256,0.010955,5
5,21.507355,0.767551,0.895340,0.029897,3,200,"{'max_features': 3, 'n_estimators': 200}",-1.519633,-1.535625,-1.526513,-1.501729,-1.517987,-1.520298,0.011173,7
6,54.836219,1.651053,2.260829,0.111713,3,500,"{'max_features': 3, 'n_estimators': 500}",-1.519772,-1.535206,-1.525958,-1.501852,-1.518147,-1.520187,0.010949,2
7,98.718885,29.816307,3.481257,1.134294,3,1000,"{'max_features': 3, 'n_estimators': 1000}",-1.520184,-1.535286,-1.526158,-1.501734,-1.517834,-1.520239,0.011042,3


In [160]:
pd.concat([pd.DataFrame(grid_rf.cv_results_["params"]),
           pd.DataFrame(grid_rf.cv_results_["mean_test_score"], 
                        columns=["neg_mean_absolute_error"])],axis=1).sort_values('neg_mean_absolute_error', ascending=False)

,max_features,n_estimators,neg_mean_absolute_error
0,2,100,-1.520174
6,3,500,-1.520187
7,3,1000,-1.520239
1,2,200,-1.520251
4,3,100,-1.520256
3,2,1000,-1.520284
5,3,200,-1.520298
2,2,500,-1.520327


Vemos que en general los valores son muy similares, nos quedamos entonces con el caso que requiere menos tiempo computacional.

Analizamos la métrica calculada (neg_mean_absolute_error) para casos TR y TS

In [161]:
nmae_tr = grid_rf.score(x_tr, y_tr)
nmae_ts = grid_rf.score(x_ts, y_ts)

In [163]:
print(f"NMAE para los datos TR = {nmae_tr:0.3f}")
print(f"NMAE para los datos TS = {nmae_ts:0.3f}")

NMAE para los datos TR = -1.476
NMAE para los datos TS = -1.509


Estos valores son ligeramente mayores a lo que se obtiene usando cross validation (cv).

El NMAE es el opuesto del MAE. El error negativo es particularmente útil para encontrar el mejor algoritmo en comparaciones múltiples con GridSearchCV(). Esto se debe a que esta función ordenar como mejor estimador al algoritmo con mayor valor para su función error. Así, el que tiene mayor NMAE será el menos negativo, y será también el MAE más bajo.

Construimos un RandomForestRegressor con los parámetros calculados

In [171]:
rf.max_features = 2
rf.n_estimators = 100

In [172]:
rf.fit(x_tr, y_tr)

RandomForestRegressor(max_features=2)

Calculamos las predicciones

In [178]:
pred_tr = rf.predict(x_tr)
pred_ts = rf.predict(x_ts)

Calculamos el score r2

In [175]:
rf.score(x_tr, y_tr)

0.2265948552460718

In [174]:
rf.score(x_ts, y_ts)

0.17975326293060712

Calculamos el score r

In [176]:
explained_variance_score(y_ts, pred_ts)

0.1797541604842674

In [177]:
explained_variance_score(y_tr, pred_tr)

0.22659489341764794

Vemos que se ha mejorado la precisión respecto a lo obtenido con regresión lineal, pero sigue siendo un modelo con baja precisión.

Agregamos los parámetros: min_samples_leaf=20, min_samples_split=40. 

In [204]:
rf = RandomForestRegressor(n_estimators=100, 
                           max_features=2, 
                           min_samples_leaf=20, 
                           min_samples_split=40)
rf.fit(x_tr, y_tr)
pred_tr = rf.predict(x_tr)
pred_ts = rf.predict(x_ts)

In [205]:
rf.score(x_tr, y_tr)

0.16289679388722078

In [206]:
rf.score(x_ts, y_ts)

0.1601525249295096

Vemos que hay un mejor acuerdo entre el score para TR y TS, por lo que el modelo es más estable

3) Modificamos el parámetro max_depth

In [233]:
rf = RandomForestRegressor(n_estimators=100, 
                           max_features=2, 
                           max_depth=100,
                           min_samples_leaf=20, 
                           min_samples_split=40,
                           random_state=0)
rf.fit(x_tr, y_tr)
pred_tr = rf.predict(x_tr)
pred_ts = rf.predict(x_ts)
print(rf.score(x_tr, y_tr))
print(rf.score(x_ts, y_ts))

0.1626693076709692
0.16001001995956488


Obtenemos un modelo ligeramente mejor. Sin embargo sigue sin dar buena precisión. Esto puede corregirse tratando adecuadamente los outliers.

4) El parámetro min_samples_leaf indica el número mínimo de muestras en un nodo terminal (o nodo externo), mientras que min_samples_split indica el número mínimo de muestras que deben haber para que un nodo interno se divida en dos. Si min_samples_leaf > min_samples_split, por más que un nodo interno pueda dividirse, no lo hará pues las muestras en sus nodos hijos deberán ser mayores que en el mismo, lo cual no puede ocurrir.